In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os


In [25]:
csv_path = "/content/house_data.csv"
df = pd.read_csv(csv_path)

# Filter for only image_id 71 and 72
df = df[df['image_id'].isin([71, 72])]

# Separate features and target
X_tab = df[['bed', 'bath', 'sqft']]  # you can add more columns
y = df['price']


In [26]:
scaler = StandardScaler()
X_tab_scaled = scaler.fit_transform(X_tab)


In [27]:
image_dir = "/content/drive/MyDrive/imgs"
image_paths = [os.path.join(image_dir, f"{img_id}.jpg") for img_id in df['image_id']]

images = []
for path in image_paths:
    img = load_img(path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    images.append(img_array)

images = np.array(images)


In [28]:
cnn_base = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')
cnn_base.trainable = False  # freeze weights

img_input = layers.Input(shape=(224, 224, 3))
x = cnn_base(img_input)
x = layers.Dense(128, activation='relu')(x)
cnn_model = models.Model(inputs=img_input, outputs=x)


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [29]:
tab_input = layers.Input(shape=(X_tab_scaled.shape[1],))
combined = layers.concatenate([cnn_model.output, tab_input])
x = layers.Dense(64, activation='relu')(combined)
x = layers.Dense(32, activation='relu')(x)
output = layers.Dense(1)(x)

model = models.Model(inputs=[cnn_model.input, tab_input], outputs=output)
model.compile(optimizer='adam', loss='mse')


In [30]:
model.fit([images, X_tab_scaled], y, epochs=10, verbose=1)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 18s 18s/step - loss: 47413116928.0000
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 519ms/step - loss: 47412854784.0000
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 599ms/step - loss: 47412740096.0000
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step - loss: 47412596736.0000
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step - loss: 47412412416.0000
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step - loss: 47412195328.0000
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - loss: 47411916800.0000
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step - loss: 47411855360.0000
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step - loss: 47411474432.0000
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - loss: 47411159040.0000


In [31]:
y_pred = model.predict([images, X_tab_scaled])
mae = mean_absolute_error(y, y_pred)
rmse = np.sqrt(mean_squared_error(y, y_pred))

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
MAE: 216995.09375
RMSE: 217740.359437565
